In [2]:
import numpy as np
import pandas as pd
import pickle
import os
from plotnine import *

%matplotlib inline

In [ ]:
# data from Train
# two models:
p1 = '../experiment_test_by_cell_line/features_monotherapy_moa_geneset_molecular_synthetic_lethality_target_gene_network_chemical_structure_drug_name/'
p2 = '../experiment_test_by_indication/features_monotherapy_moa_geneset_molecular_synthetic_lethality_target_gene_network_chemical_structure_drug_name/'
df1_aoc = pd.read_csv(p1+'/Train/SHAP_summary_aoc.csv')
df1_bliss = pd.read_csv(p1+'/Train/SHAP_summary_bliss.csv')
df2_aoc = pd.read_csv(p2+'/Train/SHAP_summary_aoc.csv')
df2_bliss = pd.read_csv(p2+'/Train/SHAP_summary_bliss.csv')

In [5]:
# show top 20 features of each category
df1_aoc['feature_type'].unique()

array(['Monotherapy drug efficacy (AOC)', 'Geneset', 'Drug name',
       'Synthetic lethality', 'Molecular_exp', 'Molecular_ddr',
       'Molecular_lof_pat', 'Molecular_coh_pat', 'Molecular_lof',
       'Molecular_cnv', 'Mode-of-action', 'Molecular_snv'], dtype=object)

In [3]:
os.makedirs('../surrogate_models', exist_ok = True)

"""
generate surrogate models for each score and each model version
"""
model_path = 'features_monotherapy_moa_geneset_molecular_synthetic_lethality_target_gene_network_chemical_structure_drug_name/'
for cv in ['../experiment_test_by_cell_line/', '../experiment_test_by_indication/']:
    out_p = '../surrogate_models'+cv.replace('.','')+model_path
    print(out_p)
    os.makedirs(out_p, exist_ok = True)
    for score in ['aoc', 'bliss']:
        df_shap = pd.read_csv(cv+model_path+'/Train/SHAP_summary_'+score+'.csv')
        print(df_shap['feature'].to_list()[:50])
        #df_shap_molecular = df_shap[df_shap['feature_type'].str.startswith("Molecular")].dropna()[:20]
        #print(df_shap_molecular['feature'].to_list())

        for feature_type in ['Molecular_exp', 'Molecular_ddr', 'Geneset', 'Synthetic lethality']:
            # print top 50 features
            #df_tmp = df_shap[df_shap['feature_type']==feature_type].dropna()[:50]
            # reorder the features by mean SHAP value
            #df_tmp = df_tmp.assign(feature = pd.Categorical(df_tmp['feature'], categories=df_tmp['feature'].to_list()[::-1]))
            df_shap_f= df_shap[df_shap['feature_type']==feature_type].dropna().sort_values(by = 'mean|SHAP val|', ascending = False)
            print(df_shap_f['feature'].to_list()[:10]) 
            #print(score, feature_type, df_shap_f.shape)
            #ggplot(df_tmp, aes(x = 'mean|SHAP val|', y = 'feature')) + geom_point()+facet_wrap(['feature_type'], scales = 'free_y')
            if feature_type == 'Molecular_exp':
                n_tops = [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150, 250, 500, 1000, 2000]
            elif feature_type == 'Molecular_ddr':
                n_tops = [5, 10, 20, 25]
            elif feature_type == 'Geneset':
                n_tops = [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150, 200]
            elif feature_type == 'Synthetic lethality':
                n_tops = [5, 10, 15, 20, 25, 30, 40, 50, 80, 100, 200, 500]   
            for n in n_tops:
                df_tmp = df_shap_f[:n]
                new_features = df_tmp['feature'].to_list()   
                pickle.dump(new_features, open(out_p+score+'_surrogate_'+feature_type+'_'+str(n)+'.pkl', 'wb'))

./surrogate_models/experiment_test_by_cell_line/features_monotherapy_moa_geneset_molecular_synthetic_lethality_target_gene_network_chemical_structure_drug_name/
['Treatment_1_ave', 'Treatment_2_ave', 'Geneset_ddr_pathway_dna_replication_dna_topoisomerase', 'Geneset_netpath_EGFR1', 'Geneset_pid_Signaling_events_mediated_by_Stem_cell_factor_receptor_c_Kit', 'Geneset_corum_DNA_synthesome_complex_17_subunits', 'Geneset_kegg_endocytosis', 'Treatment_1_RDK_1361', 'Treatment_2_RDK_1361', 'Treatment_2_RDK_833', 'Treatment_1_RDK_833', 'Treatment_2_RDK_1270', 'Geneset_PhosphoSitePlus_targets_of_PRKCA', 'Treatment_2_RDK_1494', 'Geneset_msigDB_chromatin_binding', 'Treatment_2_RDK_1372', 'Treatment_1_RDK_1494', 'Treatment_1_RDK_1270', 'Treatment_1_RDK_1372', 'Synleth_max', 'PTPRK_exp', 'Treatment_1_FP4_301', 'Synleth_BCL2L2_PIK3CG', 'Geneset_Metabase_Cell_cycle_Regulation_of_G1_S_transition_part_1', 'Geneset_Metabase_Immune_response_IL_17_signaling_pathways', 'Geneset_literature_IGF_18757322', 'Gen

In [2]:
model_path = 'features_monotherapy_moa_geneset_molecular_synthetic_lethality_target_gene_network_chemical_structure_drug_name/'
cv = '../experiment_test_by_cell_line/'
score ='aoc'
df_shap = pd.read_csv(cv+model_path+'/Train/SHAP_summary_'+score+'.csv')

In [4]:
df_shap.value_counts('feature_type')

feature_type
Geneset                            9449
Chemical_Structure_RDK             4096
Molecular_exp                      2725
Molecular_lof                      2647
Molecular_cnv                      2569
Chemical_Structure_FP2             2048
Chemical_Structure_Morgan          2048
Molecular_snv                      1740
Synthetic lethality                1122
Chemical_Structure_FP4              616
Molecular_lof_pat                   451
Chemical_Structure_MACCS            334
Molecular_coh_pat                   252
Drug name                           178
Mode-of-action                      124
Chemical_Structure_FP3              112
Molecular_ddr                        79
Monotherapy drug efficacy (AOC)       2
dtype: int64

In [11]:
# TODO: global biomarkers: analyse the overlapp of top features between test by cell line and test by indication


cv1 = '../experiment_test_by_cell_line/'
cv2 = '../experiment_test_by_indication/'

out_p1 = './surrogate_models'+cv1.replace('.','')+model_path
out_p2 = './surrogate_models'+cv2.replace('.','')+model_path
for score in ['aoc', 'bliss']:
    for feature_type in ['Molecular_exp', 'Molecular_ddr', 'Geneset', 'Synthetic lethality']:
        if feature_type == 'Molecular_exp':
            n_tops = [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150, 250, 500, 1000, 2000]
        elif feature_type == 'Molecular_ddr':
            n_tops = [5, 10, 20, 25]
        elif feature_type == 'Geneset':
            n_tops = [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150, 200]
        elif feature_type == 'Synthetic lethality':
            n_tops = [5, 10, 15, 20, 25, 30, 40, 50, 80, 100, 200, 500]   
        for n in n_tops:
            features1 = pickle.load(open(out_p1+score+'_surrogate_'+feature_type+'_'+str(n)+'.pkl', 'rb'))
            features2 = pickle.load(open(out_p2+score+'_surrogate_'+feature_type+'_'+str(n)+'.pkl', 'rb'))
            print(score, feature_type, n, len(set(features1).intersection(set(features2))))





aoc Molecular_exp 5 5
aoc Molecular_exp 10 8
aoc Molecular_exp 20 17
aoc Molecular_exp 30 27
aoc Molecular_exp 40 34
aoc Molecular_exp 50 41
aoc Molecular_exp 60 47
aoc Molecular_exp 70 57
aoc Molecular_exp 80 65
aoc Molecular_exp 90 73
aoc Molecular_exp 100 77
aoc Molecular_exp 150 123
aoc Molecular_exp 250 199
aoc Molecular_exp 500 398
aoc Molecular_exp 1000 786
aoc Molecular_exp 2000 1719
aoc Molecular_ddr 5 4
aoc Molecular_ddr 10 7
aoc Molecular_ddr 20 14
aoc Molecular_ddr 25 20
aoc Geneset 5 5
aoc Geneset 10 10
aoc Geneset 20 17
aoc Geneset 30 28
aoc Geneset 40 38
aoc Geneset 50 46
aoc Geneset 60 53
aoc Geneset 70 61
aoc Geneset 80 72
aoc Geneset 90 80
aoc Geneset 100 89
aoc Geneset 150 136
aoc Geneset 200 176
aoc Synthetic lethality 5 5
aoc Synthetic lethality 10 9
aoc Synthetic lethality 15 14
aoc Synthetic lethality 20 18
aoc Synthetic lethality 25 23
aoc Synthetic lethality 30 27
aoc Synthetic lethality 40 36
aoc Synthetic lethality 50 46
aoc Synthetic lethality 80 70
aoc Synt

In [6]:
# INTERSECT THE TOP FEATURES OF TRAIN AND TEST
model_path = 'features_monotherapy_moa_geneset_molecular_synthetic_lethality_target_gene_network_chemical_structure_drug_name/'
for cv in ['../experiment_test_by_cell_line/', '../experiment_test_by_indication/']:
    out_p = '../surrogate_models'+cv.replace('.','')+model_path
    print(out_p)
    os.makedirs(out_p, exist_ok = True)
    for score in ['aoc', 'bliss']:
        df_shap_train = pd.read_csv(cv+model_path+'/Train/SHAP_summary_'+score+'.csv')
        df_shap_test = pd.read_csv(cv+model_path+'/Test/SHAP_summary_'+score+'.csv')

        for feature_type in ['Molecular_exp', 'Molecular_ddr', 'Geneset', 'Synthetic lethality']:
            df_shap_f_train= df_shap_train[df_shap_train['feature_type']==feature_type].dropna().sort_values(by = 'mean|SHAP val|', ascending = False)
            df_shap_f_test= df_shap_test[df_shap_test['feature_type']==feature_type].dropna().sort_values(by = 'mean|SHAP val|', ascending = False)

            #print(score, feature_type, df_shap_f.shape)
            #ggplot(df_tmp, aes(x = 'mean|SHAP val|', y = 'feature')) + geom_point()+facet_wrap(['feature_type'], scales = 'free_y')
            if feature_type == 'Molecular_exp':
                n_tops = [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150, 250, 500, 1000, 2000]
            elif feature_type == 'Molecular_ddr':
                n_tops = [5, 10, 20, 25]
            elif feature_type == 'Geneset':
                n_tops = [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150, 200]
            elif feature_type == 'Synthetic lethality':
                n_tops = [5, 10, 15, 20, 25, 30, 40, 50, 80, 100, 200, 500]   
            for n in n_tops:
                df_tmp_train = df_shap_f_train[:n]
                df_tmp_test = df_shap_f_test[:n]
                new_features_train = df_tmp_train['feature'].to_list()
                new_features_test = df_tmp_test['feature'].to_list()
                # intersect the top features of train and test
                new_features = list(set(new_features_train).intersection(set(new_features_test)))
                print(score, feature_type, n, len(new_features), new_features[:10])
                



./surrogate_models/experiment_test_by_cell_line/features_monotherapy_moa_geneset_molecular_synthetic_lethality_target_gene_network_chemical_structure_drug_name/
aoc Molecular_exp 5 4 ['PKIB_exp', 'PTPRK_exp', 'TRIB1_exp', 'ADORA2A_exp']
aoc Molecular_exp 10 7 ['PKIB_exp', 'PIK3CA_exp', 'TRIB1_exp', 'PTPRK_exp', 'ADORA2A_exp', 'LIG4_exp', 'ATR_exp']
aoc Molecular_exp 20 15 ['PKIB_exp', 'PIK3CA_exp', 'LPCAT3_exp', 'AKT1_exp', 'TRIB1_exp', 'FGFR4_exp', 'PTPRK_exp', 'TYK2_exp', 'RINT1_exp', 'TAOK1_exp']
aoc Molecular_exp 30 24 ['EZH2_exp', 'PKIB_exp', 'PIK3CA_exp', 'PTPRK_exp', 'ATM_exp', 'NBN_exp', 'MKI67_exp', 'ADORA2A_exp', 'USP1_exp', 'BRCA1_exp']
aoc Molecular_exp 40 31 ['EZH2_exp', 'PKIB_exp', 'PIK3CA_exp', 'TET2_exp', 'JAK2_exp', 'PTPRK_exp', 'ATM_exp', 'NBN_exp', 'HIST1H4I_exp', 'MKI67_exp']
aoc Molecular_exp 50 36 ['EZH2_exp', 'PKIB_exp', 'PIK3CA_exp', 'TET2_exp', 'JAK2_exp', 'FLG_exp', 'PTPRK_exp', 'ATM_exp', 'NBN_exp', 'HIST1H4I_exp']
aoc Molecular_exp 60 42 ['EZH2_exp', 'PKIB_e